In [1]:
import requests
import lxml.html
import lxml.etree
import re
import pandas as pd


resp = requests.get('http://www.st-petersburg.vybory.izbirkom.ru/region/region/st-petersburg?action=show&root=1&tvd='
                    '27820001217417&vrn=27820001217413&region=78&global=&sub_region=78&prver=0&pronetvd=null&vibid='
                    '27820001217417&type=222')

tree = lxml.html.fromstring(resp.text)

tik_names = tree.xpath('//a[@style="text-decoration: none"]/text()')  # получаем названия ТИК
tik_links = tree.xpath('//a[@style="text-decoration: none"]/@href')  # получаем ссылки на ТИК


def rename(expr):
    if expr[1] == 'Территориальная избирательная комиссия':
        return 'tec' + str(expr[2])
    elif expr[1] == 'УИК':
        return str(expr[2])
    elif expr[0] == 'Цифровые избирательные участки':
        return 'dec'


tik_names = list(map(lambda x: re.sub(r'(Территориальная избирательная комиссия) №(\d+)'
                                      r'|Цифровые избирательные участки',
                                      rename, x), tik_names))

for link_num, tik_link in enumerate(tik_links):
    print(link_num, tik_link)
    resp = requests.get(tik_link)
    tree = lxml.html.fromstring(resp.text)
    if link_num < 30:
        table = tree.xpath('//table[@style="width:100%;border-color:#000000"]/tr')  # нашли таблицу
        tds = table[0].getchildren()  # берём обе половинки таблицы
        if not link_num:  # если в первый раз, то надо собрать названия полей
            tree = lxml.etree.ElementTree(tds[0])  # взяли первую
            trs = tree.xpath('//tr')  # получаем ряды таблицы
            field_keys_list = ['ec']  # список полей таблицы
            for tr in trs:  # проходимся по рядам таблицы, собирая названия полей
                fields = tr.getchildren()
                tree = lxml.etree.ElementTree(fields[1])
                field_text = tree.xpath('//text()')[0]
                field_keys_list.append(field_text)
            field_keys_list[1] = 'lec'
            field_keys_list.pop(-4)
            # закончили с первой половиной, приступаем ко второй
        field_rows_list = [[tik_names[link_num]]]  # список списков (данных) таблицы
        tree = lxml.etree.ElementTree(tds[1])
        trs = tree.xpath('//tr')
        for tr in trs:  # проходимся по рядам таблицы, собирая данные
            field_values_list = []
            fields = tr.getchildren()
            for field in fields:
                tree = lxml.etree.ElementTree(field)
                field_text = tree.xpath('//text()')
                try:
                    field_text = int(field_text[0])
                except ValueError:
                    field_text = field_text[0]
                field_values_list.append(field_text)
            field_rows_list.append(field_values_list.copy())
        field_rows_list.pop(-4)
        field_rows_list[0] *= len(field_values_list)  # дублируем название ТИК, реализуя соотношение one to many
        
    else:  # у страницы цифровых избирательных участков другая разметка, спарсим её отдельно
        table = tree.xpath('//table[@cellpadding="2"][@border="0"][@bgcolor="#ffffff"][@cellspacing="1"]')
        # нашли таблицу
        trs = table[0].getchildren()  # получаем ряды таблицы
        field_rows_list = [[tik_names[link_num]], '-']  # список списков (данных) таблицы
        for tr in trs:  # проходимся по рядам таблицы, собирая названия полей
            field_values_list = []
            fields = tr.getchildren()
            tree = lxml.etree.ElementTree(fields[-1])
            field_text = tree.xpath('//text()')[0]
            try:
                field_text = int(field_text)
            except ValueError:
                field_text = field_text
            field_values_list.append(field_text)
            field_rows_list.append(field_values_list.copy())
        field_rows_list.pop(-4)

    #  заменим названия УИК только на цифры
    field_rows_list[1] = list(map(lambda x: re.sub(r'([а-яА-Я]{3}) №(\d+)', rename, x), field_rows_list[1]))
    temp_dict = dict(zip(field_keys_list, field_rows_list))  # составляем буферный словарь из сгенерированных списков

    if not link_num:  # если в первый раз, то задаём конечный словарь
        data_dict = temp_dict.copy()
    if link_num:  # если не в первый раз, то просто дописываем данные в словарь
        for key in data_dict.keys():
            data_dict[key].extend(temp_dict[key])

df = pd.DataFrame.from_dict(data_dict)
df = df.rename(columns={'ТИК': 'TIK', 'УИК': 'UIK',
                        'Число избирателей, внесенных в список избирателей на момент окончания голосования':
                            'total_voters',
                        'Число избирательных бюллетеней, полученных участковой избирательной комиссией':
                            'total_ballots',
                        'Число избирательных бюллетеней, выданных избирателям в помещении для голосования в день '
                        'голосования': 'local_ballots',
                        'Число избирательных бюллетеней, выданных избирателям, проголосовавшим вне помещения '
                        'для голосования': 'distant_ballots',
                        'Число погашенных избирательных бюллетеней': 'unused_ballots',
                        'Число избирательных бюллетеней, содержащихся в переносных ящиках для голосования':
                        'in_mobile_boxes',
                        'Число избирательных бюллетеней, содержащихся в стационарных ящиках для голосования':
                            'in_stationary_boxes',
                        'Число недействительных избирательных бюллетеней': 'spoiled_ballots',
                        'Число действительных избирательных бюллетеней': 'valid_ballots',
                        'Число утраченных избирательных бюллетеней': 'lost_ballots',
                        'Число избирательных бюллетеней, не учтенных при получении': 'unaccounted_ballots',
                        'Амосов Михаил Иванович': 'amosov', 'Беглов Александр Дмитриевич': 'beglov',
                        'Тихонова Надежда Геннадьевна': 'tikhonova'})

# добавим недостающие поля (проще так, чем парсить)
df['amosov_share'] = df['amosov'] / (df['valid_ballots'] + df['spoiled_ballots']) * 100
df['beglov_share'] = df['beglov'] / (df['valid_ballots'] + df['spoiled_ballots']) * 100
df['tikhonova_share'] = df['tikhonova'] / (df['valid_ballots'] + df['spoiled_ballots']) * 100
df['turnout(%)'] = (df['distant_ballots'] + df['local_ballots']) / df['total_voters'] * 100

df.to_csv(index=False, path_or_buf='data.csv')
df

0 http://www.st-petersburg.vybory.izbirkom.ru/region/region/st-petersburg?action=show&tvd=27820001217417&vrn=27820001217413&region=78&global=&sub_region=78&prver=0&pronetvd=null&vibid=27820001217419&type=222
1 http://www.st-petersburg.vybory.izbirkom.ru/region/region/st-petersburg?action=show&tvd=27820001217417&vrn=27820001217413&region=78&global=&sub_region=78&prver=0&pronetvd=null&vibid=27820001217420&type=222
2 http://www.st-petersburg.vybory.izbirkom.ru/region/region/st-petersburg?action=show&tvd=27820001217417&vrn=27820001217413&region=78&global=&sub_region=78&prver=0&pronetvd=null&vibid=27820001217421&type=222
3 http://www.st-petersburg.vybory.izbirkom.ru/region/region/st-petersburg?action=show&tvd=27820001217417&vrn=27820001217413&region=78&global=&sub_region=78&prver=0&pronetvd=null&vibid=27820001217422&type=222
4 http://www.st-petersburg.vybory.izbirkom.ru/region/region/st-petersburg?action=show&tvd=27820001217417&vrn=27820001217413&region=78&global=&sub_region=78&prver=0&pron

,ec,lec,total_voters,total_ballots,local_ballots,distant_ballots,unused_ballots,in_mobile_boxes,in_stationary_boxes,spoiled_ballots,valid_ballots,lost_ballots,unaccounted_ballots,amosov,beglov,tikhonova,amosov_share,beglov_share,tikhonova_share,turnout(%)
0,tec1,1,1803,1200,587,11,602,11,587,23,575,0,0,110,345,120,18.394649,57.692308,20.066890,33.166944
1,tec1,2,1466,1100,433,14,653,14,430,19,425,0,0,53,326,46,11.936937,73.423423,10.360360,30.491132
2,tec1,3,2092,1600,576,22,1002,22,576,15,583,0,0,155,332,96,25.919732,55.518395,16.053512,28.585086
3,tec1,4,1056,1000,318,4,678,4,318,13,309,0,0,67,171,71,20.807453,53.105590,22.049689,30.492424
4,tec1,5,1827,1400,495,8,897,8,493,16,485,0,0,137,266,82,27.345309,53.093812,16.367265,27.531472
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017,tec30,2351,132,150,86,27,37,27,86,2,111,0,0,7,91,13,6.194690,80.530973,11.504425,85.606061
2018,tec30,2352,320,350,217,23,110,23,217,4,236,0,0,20,193,23,8.333333,80.416667,9.583333,75.000000
2019,tec30,2355,485,500,335,12,153,12,335,2,345,0,0,23,290,32,6.628242,83.573487,9.221902,71.546392
2020,tec30,2356,620,650,210,215,225,215,210,3,422,0,0,39,345,38,9.176471,81.176471,8.941176,68.548387
